### Your name:

<pre> Your name here</pre>

### Collaborators:

<pre> Enter the name of the people you worked with if any</pre>


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Open the housing data


In [79]:
import os
import tarfile

HOUSING_PATH = "."

def fetch_housing_data(housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()


def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()





FileNotFoundError: [Errno 2] No such file or directory: '.\\housing.tgz'

### I have the .csv file in my current working directory (have downloaded it from Canvas):

In [42]:
 """  Since we have the file ready in .csv format(provided by course instructor) and I have not downloaded/fetched ...
it using URL in tgz format from github, there is no need to use the first function
"""
HOUSING_PATH = "./"  ### I have the .csv file in my current working directory

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()



#### OR only use the following codes (but  put your files ready in your current working dir):

housing=pd.read_csv('./housing.csv')
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Build full pipeline for the data analysis following the example of the notebook.
 Hint: the main part requested to change is the algorithm used (KNN regression)


#### Considerations for building pipeline:

- Make your notebook as compact as possible. 
- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform KNN regression using a variety of values for n_neighbors (K) between 1 and 10 and both "uniform" and "distance" weights via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

In [ ]:




### Separating Labels (y) and data(x):
housing_labels=housing['median_house_value']
housing_data=housing.drop('median_house_value', axis=1)



#### Splitting train_and_test data:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train,y_test=train_test_split(housing_data, housing_labels, test_size=0.2)




###### For Training dataset:




###first we need to have the data with numerical values:
x_train_num=x_train.drop('ocean_proximity', axis=1)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

### Pipeline for numerical part of data
num_Pipeline=Pipeline([('imputer', SimpleImputer(strategy='median')), ('std_scaler', StandardScaler())])


### Pipeline for numerical and categorical (adding OneHotEncoder to pipeline)
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

full_pipeline=ColumnTransformer([('num', num_Pipeline, list(x_train_num)),('cat',OneHotEncoder(), ['ocean_proximity'])])

final_housing=full_pipeline.fit_transform(x_train)

x_train=final_housing




- ####  For Test Dataset_Preprocessing Pipeline:
##### calculation of the median of train_dataset for using in test dataset (for missing values)

In [18]:


median_of_total_bedroom_in_train=x_train_num.median().values[4]

num_Pipeline=Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value=median_of_total_bedroom_in_train)), 
                       ('std_scaler', StandardScaler())])

full_pipeline=ColumnTransformer([('num', num_Pipeline, list(x_train_num)),('cat',OneHotEncoder(), ['ocean_proximity'])])

final_housing_test=full_pipeline.fit_transform(x_test)

x_test=final_housing_test

### OR Use the train dataset Pipeline for test dataset directly (do not run the codes in the upper box):

In [58]:
x_test=full_pipeline.transform(x_test)
x_test

array([[ 0.69670507, -0.73150458,  0.58640403, ...,  0.        ,
         0.        ,  0.        ],
       [-1.46300096,  1.0477511 ,  0.50695295, ...,  0.        ,
         1.        ,  0.        ],
       [-1.34801661,  1.24440567, -0.36700895, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.22303362,  0.79022725, -0.20810679, ...,  0.        ,
         0.        ,  0.        ],
       [-1.27302682,  0.96815282,  0.66585511, ...,  0.        ,
         1.        ,  0.        ],
       [-1.35301593,  1.06179785,  1.85762135, ...,  0.        ,
         1.        ,  0.        ]])

In [68]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
param_grid=[{'n_neighbors':list(range(1,11)), 'weights':['uniform', 'distance']}]
regressor=KNeighborsRegressor()
grid_search=GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error')


- preparing the shape of labels for the model:

In [69]:
a=np.asarray(y_train).reshape(-1,1)
a.shape


(16512, 1)

In [71]:
grid_search.fit(x_train, a)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform', 'distance']}],
             scoring='neg_mean_squared_error')

- Best model is :

In [72]:
grid_search.best_estimator_

KNeighborsRegressor(n_neighbors=10, weights='distance')

In [73]:
grid_search.best_params_

{'n_neighbors': 10, 'weights': 'distance'}

##### Best root_mean_square_error value for best_estimator:

In [78]:
best_rmse=np.sqrt(-1*grid_search.best_score_)
best_rmse

59865.31957744741

In [75]:
grid_search.cv_results_

{'mean_fit_time': array([0.0222002 , 0.02039099, 0.02151408, 0.02260184, 0.02072186,
        0.020854  , 0.02140903, 0.02184939, 0.01983776, 0.02068982,
        0.0206543 , 0.01999555, 0.02000761, 0.02022333, 0.02073765,
        0.02127686, 0.02193022, 0.02036386, 0.01994338, 0.02048597]),
 'std_fit_time': array([2.40633764e-03, 5.46463601e-04, 1.40365691e-03, 2.44971759e-03,
        7.41482968e-04, 8.31193177e-04, 1.02571619e-03, 2.49455760e-03,
        4.02068504e-04, 1.42691893e-03, 7.42468310e-04, 7.02536016e-05,
        1.07378392e-05, 7.46750396e-04, 1.04540047e-03, 2.43904716e-03,
        2.37616238e-03, 3.79060361e-04, 5.03697754e-04, 7.82229821e-04]),
 'mean_score_time': array([0.0476058 , 0.04672961, 0.05533738, 0.05725489, 0.066752  ,
        0.0654305 , 0.07740126, 0.07611971, 0.0754159 , 0.08125944,
        0.08504653, 0.07981935, 0.09572792, 0.08710532, 0.09255648,
        0.09159365, 0.09829664, 0.0989532 , 0.09931493, 0.09877443]),
 'std_score_time': array([0.00306854, 

### Conclusions
For what values of n_neighbors and weight does KNeighborsRegressor perform the best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?

<pre> WRITE RESPONSE HERE </pre> 
<pre> 'n_neighbors': 10, 'weights': 'distance' </pre> 

### Read appending B

- Reflect on your last data project, read appendix B. Then, write down a few of the checklist items that your last data project could have used. If you have not yet done a data project, then write down a few of the items that you found most interesting.


<pre>Answer: I noticed that we can use the KNN for regression problems as well, and it is not only used for classification problems. </pre>

### Submit your notebook

Submit your solution to Quercus
Make sure you rename your notebook to    
W2_UTORid.ipynb    
Example W2_adfasd01.ipynb
